# Generate fieldmaps using topup

Given spin-echo EPI images with opposite phase encoding directions, this example code creates fieldmaps that can be used to correct geometrical distortions in EPI sequences like dwi or BOLD fmri.

Has been tested on sequences used on GE Signa PET/MR (UPC project) and Philips Achieva 3T (MCT-minus project).

The code is written to run in Neurodesktop.

In [1]:
#load FSL
import lmod
import os
import json
await lmod.load('fsl/6.0.5.1')
await lmod.list()
os.environ["FSLDIR"]="/cvmfs/neurodesk.ardc.edu.au/containers/fsl_6.0.5.1_20221016/fsl_6.0.5.1_20221016.simg/opt/fsl-6.0.5.1/"
os.environ["FSLOUTPUTTYPE"]="NIFTI_GZ"
os.environ["SINGULARITY_BINDPATH"]="/data,/neurodesktop-storage,/tmp,/cvmfs"

<b>Note that:</b><br>
The AP image is squashed<br>
The PA image is streched out

There seems to be some confusion with the naming of sequences, so double check this!

In [2]:
#User defined
datadir='/data/argos/Gold/UPC/MCT-studien/MR-pilot/mct/nifti'
APfile=os.path.join(datadir,'fMRI_geoCorr_blipDown.nii.gz')
PAfile=os.path.join(datadir,'fMRI_geoCorr_blipUp.nii.gz')
method='Philips'
prefix='fMRI' #prefix for output files

In [3]:
!fslinfo {APfile}
!fslinfo {PAfile}
!fslinfo {os.path.join(datadir,'fMRI_resting_210_dyns.nii.gz')}

data_type	INT16
dim1		80
dim2		80
dim3		32
dim4		3
datatype	4
pixdim1		2.823529
pixdim2		2.823529
pixdim3		3.900000
pixdim4		5.250000
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+
data_type	INT16
dim1		80
dim2		80
dim3		32
dim4		3
datatype	4
pixdim1		2.823529
pixdim2		2.823529
pixdim3		3.900000
pixdim4		5.250000
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+
data_type	INT16
dim1		80
dim2		80
dim3		32
dim4		10
datatype	4
pixdim1		2.823529
pixdim2		2.823529
pixdim3		3.900000
pixdim4		2.100000
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+


In [7]:
#define variables
ext='.nii.gz'
outdir=os.path.join(datadir,'fieldmap')
os.makedirs(outdir,exist_ok=True)
params=os.path.join(outdir,'params')
APtmp=os.path.join(outdir,'AP')
PAtmp=os.path.join(outdir,'PA')
APPAfile=os.path.join(outdir,'APPA')
outfile=os.path.join(outdir,prefix)
if method=='GE':
    config='b02b0_1.cnf'
    key='TotalReadoutTime'
elif method=='Philips':
    config='b02b0.cnf'
    key='EstimatedTotalReadoutTime'

#get parameter from json sidecar
jfile=open(APfile.split('.nii')[0] + '.json','r')
p=json.loads(jfile.read())
jfile.close()

In [5]:
%%bash -s "{p[key]}" "$params"
#create parameter file
echo "0 -1 0 $1" > $2
echo "0 1 0 $1" >> $2

In [8]:
#take last frame from AP and PA and merge
frames=!fslinfo {APfile} | grep ^dim4
frames=int(frames[0].split()[1])-1
!fslroi {APfile} {APtmp} {frames} 1
!fslroi {PAfile} {PAtmp} {frames} 1
!fslmerge -t {APPAfile} {APtmp} {PAtmp}

In [9]:
#generate fieldmap
!topup --imain={APPAfile} --datain={params} --config={config} --out={outfile} --fout={outfile+'_field'} --iout={outfile+'_unwarped'}

In [10]:
#delete temporary files
os.remove(APPAfile+ext)
os.remove(APtmp+ext)
os.remove(PAtmp+ext)

In [11]:
!cat {APPAfile+'.topup_log'}

# 	name of 4D file with images
--imain=/data/argos/Gold/UPC/MCT-studien/MR-pilot/mct/nifti/fieldmap/APPA
# name of text file with PE directions/times
--datain=/data/argos/Gold/UPC/MCT-studien/MR-pilot/mct/nifti/fieldmap/params
# 	base-name of output files (spline coefficients (Hz) and movement parameters)
--out=/data/argos/Gold/UPC/MCT-studien/MR-pilot/mct/nifti/fieldmap/fMRI
# 	name of image file with field (Hz)
--fout=/data/argos/Gold/UPC/MCT-studien/MR-pilot/mct/nifti/fieldmap/fMRI_field
# 	name of 4D image file with unwarped images
--iout=/data/argos/Gold/UPC/MCT-studien/MR-pilot/mct/nifti/fieldmap/fMRI_unwarped
# (approximate) resolution (in mm) of warp basis for the different sub-sampling levels, default 10
--warpres=20,16,14,12,10,6,4,4,4
# sub-sampling scheme, default 1
--subsamp=2,2,2,2,2,1,1,1,1
# 	FWHM (in mm) of gaussian smoothing kernel, default 8
--fwhm=8,6,4,3,3,2,1,0,0
# 	Max # of non-linear iterations, default 5
--miter=5,5,5,5,5,10,10,20,20
# Weight of regularisation,